Experimenting with multimodal LLMs to interpret graphs

In [4]:
import os
import sys
from dotenv import load_dotenv

## Add root directory to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../")))

load_dotenv()
assert os.environ["LANGCHAIN_API_KEY"], "Please set the LANGCHAIN_API_KEY environment variable"
assert os.environ["OPENAI_API_KEY"], "Please set the OPENAI_API_KEY environment variable"

In [5]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

openai_llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.environ["OPENAI_API_KEY"])
DATA_DIR = "../../../data"
image_path = DATA_DIR + "/processed/bi_dashboard_sample.png"

In [8]:
import base64

with open(image_path, "rb") as image_file:
    image_data = base64.b64encode(image_file.read()).decode("utf-8")

In [9]:
message = HumanMessage(
    content=[
        {"type": "text", "text": "Describe the charts in this dashboard."},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
    ],
)
response = openai_llm.invoke([message])
print(response.content)

The dashboard contains four distinct charts, each providing insights into different aspects of the data:

1. **Distribution of Exchange**:
   - This is a pie chart showing the percentage distribution between two exchanges: ASX and AXW. ASX accounts for 50.6% of the total, while AXW represents 49.4%. The chart visually emphasizes the near-equal distribution of transactions between these two exchanges.

2. **Distribution of Buy/Sell**:
   - A bar chart that displays the counts of buying (B) versus selling (S) transactions. The counts for both categories appear to be relatively equal, indicating a balanced trading activity between buying and selling actions.

3. **Quantity vs Price**:
   - This scatter plot depicts the relationship between quantity and price. The data points are scattered across different price levels, suggesting variability in price for different quantities. There are notable clustering patterns at specific price levels, indicating common price points for certain quantit

Add more details in the prompt

In [ ]:
categorical1, categorical2, continuous1, continuous2 = 'Exchange', 'BuySell', 'Quantity', 'Price'
var_description = """
The Exchange variable describes the stock exchange where the trade was executed. 
The BuySell variable describes whether the trade was a buy or a sell. 
The Quantity variable describes the number of shares traded. 
The Price variable describes the price at which the shares were traded.
"""